# FFWIZARD.COM --- QB and DEF Combos

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter
from datetime import datetime

### Load NFL Schedule

In [2]:
# load nfl schedule data (manually entered)
def load_schedule():
    nfl_sched = pd.read_csv('2018_NFL_Schedule.csv')
    return nfl_sched

### Defense Strength Data

In [3]:
def calc_defense_strength(bye_adjustor):

    # load nfl defense strength data (manually entered)
    def_strength_data = pd.read_csv('Defense_Strength.csv')

    #remove 'source' row
    def_strength = def_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    def_rank_weights = pd.Series([0.20, 0.25, 0.25, 0.10, 0.10, 0.10])
    
    # create weighted defense rankings array based on weights
    num_teams = len(def_strength)
    weighted_def_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_def_rank[i] = def_rank_weights[0] * float(def_strength.iloc[i, 1]) + def_rank_weights[1] * float(def_strength.iloc[i, 2]) + def_rank_weights[2] * float(def_strength.iloc[i, 3]) + def_rank_weights[3] * float(def_strength.iloc[i, 4])
    weighted_def_rank_df = pd.Series(weighted_def_rank)

    # scale weighted defense rankings to have mean of 0 and std dev of 1
    def_rank_scaled = pd.Series(preprocessing.scale(weighted_def_rank_df))

    # create table with team and score info
    defense_score_no_bye = pd.concat([def_strength.loc[:, "Team"], weighted_def_rank_df, def_rank_scaled], axis=1)
    col_names = ["Team", "Def_Score", "Def_Score_Scaled"]
    defense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    defense_score = pd.concat([defense_score_no_bye, bye_array])

    defense_score = defense_score.reset_index(drop=True)
    
    return defense_score

### Offense Strength Data

In [4]:
def calc_offense_strength(bye_adjustor):

    # load nfl offense strength data (manually entered)
    off_strength_data = pd.read_csv('Offense_Strength.csv')

    # remove 'source' row
    off_strength = off_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    off_rank_weights = pd.Series([0.3, 0.1, 0.3, 0.25, 0.05])
    
    # create weighted offense rankings array based on weights
    num_teams = len(off_strength)
    weighted_off_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_off_rank[i] = off_rank_weights[0] * float(off_strength.iloc[i, 1]) + off_rank_weights[1] * float(off_strength.iloc[i, 2]) + off_rank_weights[2] * float(off_strength.iloc[i, 3]) + off_rank_weights[3] * float(off_strength.iloc[i, 4])  + off_rank_weights[4] * float(off_strength.iloc[i, 5])
    weighted_off_rank_df = pd.Series(weighted_off_rank)

    # scale weighted offense rankings to have mean of 0 and std dev of 1
    off_rank_scaled = pd.Series(preprocessing.scale(weighted_off_rank_df))

    # create table with team and score info
    offense_score_no_bye = pd.concat([off_strength.loc[:, "Team"], weighted_off_rank_df, off_rank_scaled], axis=1)
    col_names = ["Team", "Off_Score", "Off_Score_Scaled"]
    offense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    offense_score = pd.concat([offense_score_no_bye, bye_array])

    offense_score = offense_score.reset_index(drop=True)
    
    return offense_score

In [5]:
def matchup_scores_by_week(schedule, score, last_week):

    schedule_score = schedule.copy()

    # replace schedule with actual matchup scores
    for i in range(0, schedule.shape[0]):
        for j in range(1, schedule.shape[1]):
            for k in range(0, len(score)):
                if score.loc[k, "Team"] == schedule.iloc[i, j]:
                    schedule_score.iloc[i, j] = score.iloc[k, 2]

    custom_sched_score = schedule_score.iloc[:, :(last_week+1)]

    return custom_sched_score

In [6]:
bye_adjustor = -2.5
schedule = load_schedule()
score = calc_defense_strength(bye_adjustor)
target_team_list = range(0, 32)
last_week_list = [17]


In [7]:
dummy = matchup_scores_by_week(schedule, score, last_week_list[0])

In [8]:
dummy

,Team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,ARI,1.12114,-1.58583,-0.47206,-0.287661,1.18015,-1.55632,-0.708089,1.18015,-2.5,0.0811353,1.07689,-0.796601,0.626954,0.442556,-0.221278,-1.58583,-0.287661
1,ATL,-1.50469,-0.715465,-0.767097,0.575323,-0.685962,0.789225,0.700714,-2.5,1.12114,1.32029,0.228654,-0.767097,-1.15065,0.626954,0.140143,-0.715465,0.789225
2,BAL,0.715465,0.575323,-0.708089,-0.685962,1.32029,-0.213902,-0.767097,-0.715465,-0.685962,-2.5,0.575323,1.07689,-0.221278,0.0811353,0.789225,-0.796601,1.32029
3,BUF,-1.15065,-0.796601,-1.55632,0.626954,-0.213902,-0.67121,1.72597,-0.752345,-0.47206,1.57845,-2.5,-1.79235,1.57845,1.57845,0.442556,-0.752345,1.57845
4,CAR,0.228654,-0.221278,0.575323,-2.5,0.700714,1.12114,-1.50469,-1.15065,0.789225,-0.685962,0.442556,-0.287661,0.789225,1.32029,-0.767097,-0.221278,-0.767097
5,CHI,0.626954,-0.287661,0.140143,0.789225,-2.5,1.57845,-0.752345,1.57845,0.715465,0.442556,-1.55632,0.442556,0.700714,-1.58583,0.626954,1.18015,-1.55632
6,CIN,1.72597,-1.15065,-0.715465,-0.221278,1.57845,-0.685962,0.0811353,0.789225,-2.5,-0.767097,-1.15065,1.32029,-0.708089,-0.796601,1.07689,1.32029,-0.685962
7,CLE,-0.685962,-0.767097,1.57845,1.07689,-1.15065,-0.796601,0.789225,-0.685962,0.0811353,-0.221278,-2.5,0.575323,-0.67121,-0.715465,-0.708089,0.575323,-1.15065
8,DAL,-0.715465,0.700714,-0.287661,0.442556,-0.67121,-1.79235,1.12114,-2.5,-0.213902,-1.50469,-0.221278,1.12114,-0.767097,-1.50469,1.72597,0.789225,0.700714
9,DEN,-0.287661,1.07689,-1.15065,0.0811353,1.57845,-1.58583,0.140143,0.0811353,-0.67121,-2.5,-0.796601,-0.685962,0.575323,1.18015,1.32029,1.07689,-0.796601


In [9]:
target_team = 1
target_matchups = dummy.iloc[target_team, :]

In [10]:
target_matchups

Team         ATL
1       -1.50469
2      -0.715465
3      -0.767097
4       0.575323
5      -0.685962
6       0.789225
7       0.700714
8           -2.5
9        1.12114
10       1.32029
11      0.228654
12     -0.767097
13      -1.15065
14      0.626954
15      0.140143
16     -0.715465
17      0.789225
Name: 1, dtype: object

In [11]:
def combo_score(target_team, custom_sched_score, last_week):

    target_team_name = custom_sched_score.loc[target_team, "Team"]
    col_label = target_team_name + "_" + str(last_week)

    target_matchups = custom_sched_score.iloc[target_team, :]

    combos_calc = custom_sched_score.copy()

    sum_combos_score = []

    for i in range(0, custom_sched_score.shape[0]):
        for j in range(1, custom_sched_score.shape[1]):
            if target_matchups[j] > custom_sched_score.iloc[i, j]:
                combos_calc.iloc[i, j] = 0
            else:
                combos_calc.iloc[i, j] = custom_sched_score.iloc[i, j] - target_matchups[j]

        sum_combos_score.append(combos_calc.iloc[i, 1:].sum())

    sum_combos_score_df = pd.DataFrame({col_label: sum_combos_score})

    targ_team_combo = pd.concat([custom_sched_score.loc[:, "Team"], sum_combos_score_df, custom_sched_score.iloc[:, 1:]], axis=1)

    targ_team_combo_sorted = targ_team_combo.sort_values(by=[col_label], ascending=False)
    
    return targ_team_combo_sorted, col_label

### Inputs

In [12]:
bye_adjustor = -2.5
schedule = load_schedule()
def_score = calc_defense_strength(bye_adjustor)
off_score = calc_offense_strength(bye_adjustor)

target_team_list = range(0, 32)
last_week_list = [17]

### Make Excel File with Defense Matchup Strengths

In [13]:
# make writer object for Excel file
writer = pd.ExcelWriter('./combos/qb_combos.xlsx', engine='xlsxwriter')

# iterate through every input permutation
for i in range(0, len(last_week_list)):
    
    for j in range(0, len(target_team_list)):
        
        
        custom_sched_score = matchup_scores_by_week(schedule, def_score, last_week_list[i])
        targ_team_combo_sorted, col_label = combo_score(target_team_list[j], custom_sched_score, last_week_list[i])
        
        targ_team_combo_sorted.to_excel(writer, sheet_name=col_label)
        
        print col_label

writer.save()

ARI_17
ATL_17
BAL_17
BUF_17
CAR_17
CHI_17
CIN_17
CLE_17
DAL_17
DEN_17
DET_17
GB_17
HOU_17
IND_17
JAX_17
KC_17
LAC_17
LAR_17
MIA_17
MIN_17
NE_17
NO_17
NYG_17
NYJ_17
OAK_17
PHI_17
PIT_17
SEA_17
SF_17
TB_17
TEN_17
WAS_17


### Make Excel File with Offense Matchup Strengths

In [14]:
# make writer object for Excel file
writer = pd.ExcelWriter('./combos/def_combos.xlsx', engine='xlsxwriter')

# iterate through every input permutation
for i in range(0, len(last_week_list)):
    
    for j in range(0, len(target_team_list)):
        
        
        custom_sched_score = matchup_scores_by_week(schedule, off_score, last_week_list[i])
        targ_team_combo_sorted, col_label = combo_score(target_team_list[j], custom_sched_score, last_week_list[i])
        
        targ_team_combo_sorted.to_excel(writer, sheet_name=col_label)
        
        print col_label

writer.save()

ARI_17
ATL_17
BAL_17
BUF_17
CAR_17
CHI_17
CIN_17


AttributeError: 'str' object has no attribute 'ndim'